<h1 style="text-align:center;line-height:1.5em;font-size:30px;">Transfer Learning with a Convolutional Neural Network for Hydrological Streamline Detection</h1>

<p style="text-align:center;">
    Nattapon Jaroenchai$^{a, b}$ Shaohua Wang$^{a, b}$, Li Chen$^{a, b}$, Lawrence V. Stanislawski$^{c}$, Ethan Shavers$^{c}$, E. Lynn Usery$^{c}$, Shaowen Wang$^{a, b}$
</p>

<p style="text-align:center;">
$^{a}$ Department of Geography and Geographic Information Science, University of Illinois at Urbana-Champaign, Urbana, IL, US<br>
$^{b}$ CyberGIS Center for Advanced Digital and Spatial Studies, University of Illinois at Urbana-Champaign, Urbana, IL, USA<br>
$^{c}$ U.S. Geology Survey, Center of Excellence for Geospatial Information Science, Rolla, MO, USA <br>
$^{d}$ School of Geoscience and Info-Physics, Central South University, Changsha, Hunan, China <br>
</p>

---
    
**Notebook Structure:**
- [Introduction](introduction.ipynb)
- Codes
 - [Data Preprocessing](preprocessing.ipynb)
 - [Experiment 1: different input datasets](experiment_1.ipynb)
 - [Experiment 2: retrain different part of the network](experiment_2.ipynb)
 - [Experiment 3: different sample sizes](experiment_3.ipynb)
 - [Model Evaluation](evaluation.ipynb) 
 - [Conclusion](conclusion.ipynb) 

In [ ]:
import copy
import random
import sys
import numpy as np
from sklearn.metrics import f1_score, precision_score,recall_score
import glob

buf =30

groundtruthlist = np.load('./samples/bottom_half_test_label.npy').flatten()
print(groundtruthlist.shape)

In [ ]:
experiment_num = 1

# change the file name to the prediction results file that you want to evaluate
prediction_results_file_name = "experiment_1_without_NAIP_model.npy"


prediction_to_evaluate = "./prediction_results/experiment_"+str(experiment_num)+"/"

predictionlist = np.load(path).flatten()